In [1]:
!pip install -q streamlit pyngrok google-generativeai PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00


In [2]:
from getpass import getpass
import google.generativeai as genai
from pyngrok import conf

GEMINI_KEY   = getpass("Gemini API key: ")
NGROK_TOKEN  = getpass("ngrok authtoken: ")

# keep Gemini key only in RAM
genai.configure(api_key=GEMINI_KEY)

# register the ngrok token in‑memory (no YAML file written)
conf.get_default().auth_token = NGROK_TOKEN

# make the key visible to the Streamlit subprocess only
import os, textwrap, subprocess, json, time, pathlib
os.environ["GOOGLE_API_KEY"] = GEMINI_KEY


Gemini API key: ··········
ngrok authtoken: ··········


In [7]:
app_code = """
import streamlit as st, fitz, google.generativeai as genai, os, re

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
model = genai.GenerativeModel("models/gemini-1.5-flash")

def pdf_to_chapters(file):
    doc = fitz.open(stream=file.read(), filetype='pdf')
    text = ''.join(p.get_text() for p in doc)
    # naive splitter – adjust if your PDFs have different headers
    chapters = re.split(r'\\n\\s*(?:CHAPTER|Chapter|chapter)\\s+\\d+.*', text)
    return [c.strip() for c in chapters if len(c.strip()) > 100]

def summarize(chapter):
    prompt = f\"\"\"Summarize the chapter below, list 3‑5 key insights,
    and create 5 flashcards (Q&A).  Output must follow this template:

    ### Summary:
    ...
    ### Key Insights:
    1. ...
    ### Flashcards:
    Q: ...
    A: ...
    Chapter:
    {chapter[:4000]}
    \"\"\"
    return model.generate_content(prompt).text

st.title("📚 Gemini Book Summarizer + Flashcards")
pdf = st.file_uploader("Upload a PDF book", type="pdf")

if pdf:
    st.info("Extracting chapters…")
    chapters = pdf_to_chapters(pdf)
    st.success(f"Found {len(chapters)} chapters")
    picks = st.multiselect("Select chapters to summarise",
                           range(1, len(chapters)+1), default=[1])

    if st.button("Generate"):
        full = []
        for i in picks:
            st.subheader(f"Chapter {i}")
            with st.spinner("Gemini is thinking…"):
                out = summarize(chapters[i-1])
            st.markdown(out)
            full.append(f"### Chapter {i}\\n{out}")
        st.download_button("💾 Download all", '\\n\\n'.join(full),
                           file_name="summaries.txt")
"""
pathlib.Path("app.py").write_text(app_code)


1645

In [8]:
from pyngrok import ngrok, conf
import subprocess, time, os, signal, sys, contextlib, pathlib, json

# kill stray tunnels if any
ngrok.kill()

# start Streamlit in the background (quiet logs → logs.txt)
proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.headless=true"],
    stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT
)

# wait a moment for Streamlit to boot
time.sleep(5)

# expose port 8501
public_url = ngrok.connect(8501)
print("🌐 Open your app →", public_url)


🌐 Open your app → NgrokTunnel: "https://1a4b-34-21-93-189.ngrok-free.app" -> "http://localhost:8501"
